In [73]:
# Al cargar los archivos salen algunas warnings que indican las anotaciones
# equivocadas. Las podemos omitir la libreria warnings.
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 

import numpy as np
import matplotlib.pyplot as plt
import mne

In [74]:
path = "/media/usbdisk/data/ProyectoPSG/data/PSG1.edf"

# Carga de los datos utilizando la librería MNE

Los archivos de PoliSomnoGrama se nos dan en formato `.edf`, un formato médico especial que almacena las señales junto con más información como la frecuencia de muestreo, las anotaciones de los especialistas, etc. Para poder cargarlo tenemos que utilizar la librería `mne`.

In [75]:
raw_data = mne.io.read_raw_edf(path)
raw_data

Extracting EDF parameters from /media/usbdisk/data/ProyectoPSG/data/PSG1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


<RawEDF | PSG1.edf, 50 x 13569536 (26503.0 s), ~51 kB, data not loaded>

Como podemos comprobar, esta función nos devuelve un objeto propio de la libreria que contiene mucha información. Wste objeto se puede indexar para obtener los datos, pero como podemos ver, no tienen la forma que podríamos esperar.

In [76]:
type(raw_data)

mne.io.edf.edf.RawEDF

In [77]:
a,b = raw_data[0]
a.shape, b.shape

((1, 13569536), (13569536,))

## Anotaciones

Lo más especial de este tipo de datos es que incluyen anotaciones médicas. Podemos acceder a ellas mediante el atributo `.annotations`, y al igual que antes, podemos indexarlas para ver la información:

In [78]:
raw_data.annotations

<Annotations | 3038 segments: EEG arousal (19), Hypopnea (23), Impedance ...>

Como se puede ver, contienen toda la información necesaria para trocear la señal original:

- `'onset'` indica el tiempo en el que comienza la época.
- `'duration'` indica la duración.
- `'description'` indica la etiqueta que le han puesto los médicos.
- `'orig_time'` indica la fecha en la que se toma ese registro.

In [79]:
raw_data.annotations[80]

OrderedDict([('onset', 1920.0),
             ('duration', 30.0),
             ('description', 'Sleep stage N2'),
             ('orig_time',
              datetime.datetime(2020, 1, 1, 22, 21, 19, tzinfo=datetime.timezone.utc))])

Podemos obtener las diferentes etiquetas y sus cantidades si iteramos sobre este objeto:

In [80]:
etiquetas = []
for ann in raw_data.annotations:
    etiquetas.append(ann['description'])

In [81]:
from collections import Counter

In [82]:
Counter(etiquetas)

Counter({'Montage:PR, Ref': 2,
         'Start Recording': 1,
         'Video Recording ON': 1,
         'Recording Analyzer - Sleep Events': 1,
         'Recording Analyzer - Auto-Staging': 1,
         'Recording Analyzer - ECG': 1,
         'Recording Analyzer - Data Trends': 1,
         'Limb Movement': 103,
         'Sleep stage W': 247,
         'EEG arousal': 19,
         'Lights Off': 1,
         'Started Analyzer - Sleep Events': 1,
         'Sleep stage N1': 25,
         'Sleep stage N2': 317,
         'Snoring': 1870,
         'Sleep stage N3': 193,
         'Hypopnea': 23,
         'Oxygen Desaturation': 47,
         'Obstructive Apnea': 79,
         'Sleep stage R': 101,
         'Oximeter Event': 2,
         'Pulse Rate Event': 1,
         'Impedance at 10 kOhm': 1})

## Transformación de los datos en épocas

El estudio de este tipo de datos se realiza en periodos de 30s que se etiquetan según un criterio médico. Cada uno de estos periodos se llama **Época**. Ya hemos visto que el archivo que hemos cargado tiene incluída también la información de estas etiquetas, así que la podemos aprovechar para transformar nuestros datos.

Como se puede ver en la función siguiente, este proceso tiene varias partes:

- Extraemos los diferentes eventos a partir de las anotaciones de los datos. Podemos utilizar expresiones regulares para quedarnos solamente con las fases del sueño que nos interesan. Esto es importante porque en el archivo se recogen muchas más fases de las que necesitamos y ocupan mucho espacio en memoria.
- `mne.Epochs` trocea la señal original según le indican los eventos. Nos permite elegir los canales que queremos utilizar. Si queremos utilizarlos todos basta con no poner nada.

In [83]:
def get_epochs(data, channels=None):
        sampling_rate = data.info['sfreq']
        events, events_id = mne.events_from_annotations(data, regexp='Sleep stage [A-Z]\d*')

        tmax = 30. - 1. / sampling_rate  # tmax is included
        epochs = mne.Epochs(raw=data, 
                            events=events,
                            event_id=events_id,
                            tmin=0., 
                            tmax=tmax, 
                            baseline=None, 
                            event_repeated='merge',
                            picks=channels)

        epochs.drop_bad()
        return epochs, sampling_rate

In [84]:
epochs, sr = get_epochs(raw_data)

Used Annotations descriptions: ['Sleep stage N1', 'Sleep stage N2', 'Sleep stage N3', 'Sleep stage R', 'Sleep stage W']
Not setting metadata
Not setting metadata
883 matching events found
No baseline correction applied
0 projection items activated
Loading data for 883 events and 15360 original time points ...
1 bad epochs dropped


La particularidad que tienen estas épocas es que no se pueden indexar como hacíamos antes para obtener los datos, si no que tendremos que utilizar el método `.get_data()`. Hay que notar que como este tipo de datos ocupa mucho especio, los datos de cada época se cargan cuando llamamos a este método. Podemos ver que las dimensiones que tiene cada época son `(1, Canales, Puntos por canal)`. (En realidad el primer número corresponde al número de épocas, pero como solo accedemos a una época obtenemos un 1, también se puede hacer `.get_data()` sobre el objeto completo para cargarlo entero en memoria a la vez).

In [85]:
epochs[0].get_data().shape

Loading data for 1 events and 15360 original time points ...


(1, 50, 15360)

Como no podía ser de otra manera, este objeto también almacena las etiquetas correspondientes a cada época. Por una parte tenemos el atributo `.events` que contiene las etiquetas codificadas en la última dimensión, mientras que en `.event_id` encontramos un diccionario que nos mapea estas etiquetas codificadas con su etiqueta original:

In [86]:
epochs.events

array([[   15360,        0,        5],
       [   30720,        0,        5],
       [   46080,        0,        5],
       ...,
       [13516800,        0,        5],
       [13532160,        0,        5],
       [13547520,        0,        5]])

In [87]:
epochs.event_id

{'Sleep stage N1': 1,
 'Sleep stage N2': 2,
 'Sleep stage N3': 3,
 'Sleep stage R': 4,
 'Sleep stage W': 5}

# Pasando a NumPy Array para el análisis

Finalmente podemos obtener un `array` completo con nuestros datos y otro con nuestras `etiquetas` para realizar el análisis correspondiente:

In [88]:
X = epochs.get_data()
Y = epochs.events[:,-1]
X.shape, Y.shape

Loading data for 882 events and 15360 original time points ...


((882, 50, 15360), (882,))

## Realizando operaciones entre canales

Hay que tener en cuenta que los médicos que realizan el etiquetado de los PoliSomnoGramas no ven los datos tal cual los hemos importado. Ellos solamente utilizan 9 canales y, además, realizan operaciones de referenciación entre ellos. Para facilitar este análisis y optimizar el uso de memoria podemos volver a cargar los datos pero utilizando únicamente los canales que necesitamos:

In [89]:
channels = ["C3", "C4", "A1", "A2", "O1", "O2", "LOC", "ROC", "LAT1", "LAT2", "ECGL", "ECGR", "CHIN1", "CHIN2"]

In [90]:
epochs, sr = get_epochs(raw_data, channels=channels)

Used Annotations descriptions: ['Sleep stage N1', 'Sleep stage N2', 'Sleep stage N3', 'Sleep stage R', 'Sleep stage W']
Not setting metadata
Not setting metadata
883 matching events found
No baseline correction applied
0 projection items activated
Loading data for 883 events and 15360 original time points ...
1 bad epochs dropped


Las operaciones de referenciación que utilizan son:
$$ C3 - \frac{A1+A2}{2} $$
$$ C4 - \frac{A1+A2}{2} $$
$$ 01 - \frac{A1+A2}{2} $$
$$ 02 - \frac{A1+A2}{2} $$
$$ LOC - A2 $$
$$ ROC - A1 $$
$$ LAT1 - LAT2 $$
$$ ECGL - ECGR $$
$$ CHIN1 - CHIN2 $$

Aunque son bastantes operaciones, son fáciles de hacer una vez tenemos los datos en formato `array`.

In [91]:
X = epochs.get_data()
Y = epochs.events[:,-1]
X.shape, Y.shape

Loading data for 882 events and 15360 original time points ...


((882, 14, 15360), (882,))

Podemos construir un diccionario que haga nuestro código mucho más legible para hacer estas operaciones:

In [92]:
# Aplicamos expand_dims para no perder la dimensión correspondiente a los canales
data = {ch:np.expand_dims(X[:,i,:],1) for i, ch in enumerate(channels)}

In [93]:
channel1 = data["C3"] - (data["A1"]+data["A2"])/2
channel2 = data["C4"] - (data["A1"]+data["A2"])/2
channel3 = data["O1"] - (data["A1"]+data["A2"])/2
channel4 = data["O2"] - (data["A1"]+data["A2"])/2
channel5 = data["LOC"] - data["A2"]
channel6 = data["ROC"] - data["A1"]
channel7 = data["LAT1"] - data["LAT2"]
channel8 = data["ECGL"] - data["ECGR"]
channel9 = data["CHIN1"] - data["CHIN2"]

In [94]:
X = np.concatenate([channel1, channel2, channel3, channel4, channel5, channel6, channel7, channel8, channel9],1)
X.shape

(882, 9, 15360)